# We would like to confirm that more of their resources are dedicated to their customer facing products as the day goes on

### To do this we will have to compare total resource use, by total resources dedicated to their services

It is important for us to take care when adding these that we calculate resources correctly. We will want to look at something like this for every timestamp, for every server, for ever service instance, calculate total resource Util.

In [5]:
import pandas as pd
import numpy as np

In [28]:
service_containers = pd.read_csv("/home/codyblakeney/private/personal_repo/gc_project/data/container_event.csv")

In [29]:
service_instances = pd.read_csv("/home/codyblakeney/private/personal_repo/gc_project/data/container_usage.csv")

In [30]:
ids_at_39600 = np.array(service_instances[service_instances.ts == 39600].instance_id)

In [31]:
service_containers[service_containers.instance_id == ids_at_39600[0]]

,ts,event,instance_id,machine_id,plan_cpu,plan_mem,plan_disk,cpuset,Unnamed: 8
1428,0,Create,117,823,4,0.042409,0.034085,40|41|42|43,NaN


In [33]:
service_instances[service_instances.instance_id == ids_at_39600[0]]

,ts,instance_id,cpu_util,mem_util,disk_util,load1,load5,load15,avg_cpi,avg_mpki,max_cpi,max_mpki
11,39600,117,3.66,38.299999,12.80,0.14,0.18,0.20,0.164531,0.227815,2.046043,3.524049
1607,42600,117,4.00,38.480000,12.90,0.14,0.16,0.20,0.164531,0.227815,2.046043,3.524049
20878,39900,117,3.58,38.299999,12.80,0.10,0.14,0.18,0.164531,0.227815,2.046043,3.524049
22117,42000,117,3.70,38.400002,12.84,0.30,0.24,0.20,0.164531,0.227815,2.046043,3.524049
42732,40800,117,3.70,38.360001,12.86,0.18,0.12,0.12,0.164531,0.227815,2.046043,3.524049
43092,42300,117,3.56,38.440001,12.90,0.16,0.20,0.20,0.164531,0.227815,2.046043,3.524049
62776,41400,117,3.78,38.400002,12.82,0.18,0.18,0.18,0.164531,0.227815,2.046043,3.524049
83686,41100,117,3.60,38.400002,12.90,0.18,0.20,0.18,0.164531,0.227815,2.046043,3.524049
83936,40500,117,3.72,38.380001,12.86,0.16,0.12,0.10,0.164531,0.227815,2.046043,3.524049
84297,42900,117,3.52,38.420001,12.90,0.10,0.14,0.20,0.164531,0.227815,2.046043,3.524049
